In [ ]:
import networkx as nx
import igraph as ig
import louvain

##### Create a total graph and alliance graph
NB: is necessary run this code only one time (you probably have already the net in the dataset folder); i hope wont be necessary come back on this

In [131]:
# create a multigraph for all nets type
messageGraph = nx.MultiDiGraph()
tradeGraph = nx.MultiDiGraph()
attackGraph = nx.MultiDiGraph()

# number of graph (30 in our case)
nGraph = 30

# for every day is needed create total and alliance graph
for i in range(0, nGraph):
    print("Graph number: ", (i + 1))
    
    # for every day is needed reinitialize the "real" graphs (graph with edge label)
    realAttack = nx.MultiDiGraph()
    realMessage = nx.MultiDiGraph()
    realTrade = nx.MultiDiGraph()
    
    # for every day is needed reinitialize the alliance graph and total graph
    allianceGraph = nx.MultiDiGraph()
    totalGraph = nx.MultiDiGraph()
    
    # read from Dataset/Graphml-File the correct file based on type and day
    attackGraph = nx.read_graphml("../Dataset/Graphml-File/Attacks-Network-Graphml/attacks-timestamped-2009-12-"+str(i+1)+".graphml")
    messageGraph = nx.read_graphml("../Dataset/Graphml-File/Messages-Network-Graphml/messages-timestamped-2009-12-"+str(i+1)+".graphml")
    tradeGraph = nx.read_graphml("../Dataset/Graphml-File/Trades-Network-Graphml/trades-timestamped-2009-12-"+str(i+1)+".graphml")
    
    # copy xxxxGraph in realxxxx adding label to edges; keep attentio about key, it must be unique (we use old key + 
    # letter for define edge tipology) 
    for itr in messageGraph.edges_iter(None, True, True):
        realMessage.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"m", Label= "Message")
    
    for itr in tradeGraph.edges_iter(None, True, True):
        realTrade.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"t", Label= "Trade")

    for itr in attackGraph.edges_iter(None, True, True):
        realAttack.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"a", Label= "Attack")
    
    # compose (a union) in one graph the needed graphs 
    allianceGraph = nx.compose(realMessage, realTrade)
    totalGraph = nx.compose(allianceGraph, realAttack)
    # save total graph and alliance graph in the correct file
    nx.write_graphml(totalGraph, "../Dataset/Graphml-File/Total-Network-Graphml/total-2009-12-"+str(i+1)+".graphml")
    nx.write_graphml(realMessage, "../Dataset/Graphml-File/Alliance-Network-Graphml/alliance-2009-12-"+str(i+1)+".graphml")

    # print for check, i checked that sum of graphs is the equal to alliance graph and total graph (according to the case)
    # but i think is a better that YOU MAKE THE SAME; ONE MORE CONTROL IS BETTER!
    print("ATTACK GRAPH: ", nx.info(attackGraph))
    print("MESSAGE GRAPH: ", nx.info(messageGraph))
    print("TRADE GRAPH: ", nx.info(tradeGraph))
    print("ALLIANCE GRAPH: ", nx.info(allianceGraph))
    print("TOTAL GRAPH: ", nx.info(totalGraph))
    print("") 

Graph number:  1
ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2807
Number of edges: 32997
Average in degree:  11.7553
Average out degree:  11.7553
MESSAGE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1963
Number of edges: 16346
Average in degree:   8.3271
Average out degree:   8.3271
TRADE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1336
Number of edges: 9390
Average in degree:   7.0284
Average out degree:   7.0284
ALLIANCE GRAPH:  Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 2221
Number of edges: 25736
Average in degree:  11.5876
Average out degree:  11.5876
TOTAL GRAPH:  Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 3757
Number of edges: 58733
Average in degree:  15.6330
Average out degree:  15.6330

Graph number:  2
ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2802
Number of edges: 32782
Average in degree:  11.6995
Average out degree:  11.6995
MESSAGE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2100
Number of edges: 

ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2082
Number of edges: 21628
Average in degree:  10.3881
Average out degree:  10.3881
MESSAGE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1833
Number of edges: 14447
Average in degree:   7.8816
Average out degree:   7.8816
TRADE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1145
Number of edges: 9310
Average in degree:   8.1310
Average out degree:   8.1310
ALLIANCE GRAPH:  Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 1998
Number of edges: 23757
Average in degree:  11.8904
Average out degree:  11.8904
TOTAL GRAPH:  Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 2967
Number of edges: 45385
Average in degree:  15.2966
Average out degree:  15.2966

Graph number:  13
ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2066
Number of edges: 20675
Average in degree:  10.0073
Average out degree:  10.0073
MESSAGE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1786
Number of edges: 14597
Average in

ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1732
Number of edges: 16477
Average in degree:   9.5133
Average out degree:   9.5133
MESSAGE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1632
Number of edges: 13178
Average in degree:   8.0748
Average out degree:   8.0748
TRADE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1013
Number of edges: 8062
Average in degree:   7.9585
Average out degree:   7.9585
ALLIANCE GRAPH:  Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 1765
Number of edges: 21240
Average in degree:  12.0340
Average out degree:  12.0340
TOTAL GRAPH:  Name: compose( ,  )
Type: MultiDiGraph
Number of nodes: 2610
Number of edges: 37717
Average in degree:  14.4510
Average out degree:  14.4510

Graph number:  24
ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1716
Number of edges: 16467
Average in degree:   9.5962
Average out degree:   9.5962
MESSAGE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1618
Number of edges: 11773
Average in

##### Creation of our structure for maintain all nets
Create a dictionary of lists, the key rappresents the day (start 0 end 29), the list is a list of nets about the specific day with this order: (attackGraph, messageGraph, TradeGraph, allianceGraph, totalGraph)

In [133]:
# our structure 
allNetwork = dict()

# for structure graphs inserting
attackGraph = ig.Graph()
messageGraph = ig.Graph()
tradeGraph = ig.Graph()
totalGraph = ig.Graph()
allianceGraph = ig.Graph()

for i in range(0, nGraph):
    print("Day: ", (i + 1))
    
    # read all needed graphs
    attackGraph = attackGraph.Read_GraphML("../Dataset/Graphml-File/Attacks-Network-Graphml/attacks-timestamped-2009-12-"+str(i+1)+".graphml")
    messageGraph = messageGraph.Read_GraphML("../Dataset/Graphml-File/Messages-Network-Graphml/messages-timestamped-2009-12-"+str(i+1)+".graphml")
    tradeGraph = tradeGraph.Read_GraphML("../Dataset/Graphml-File/Trades-Network-Graphml/trades-timestamped-2009-12-"+str(i+1)+".graphml")
    allianceGraph = allianceGraph.Read_GraphML("../Dataset/Graphml-File/Alliance-Network-Graphml/alliance-2009-12-"+str(i+1)+".graphml")
    totalGraph = totalGraph.Read_GraphML("../Dataset/Graphml-File/Total-Network-Graphml/total-2009-12-"+str(i+1)+".graphml")
    
    # put as a list in a dictionary 
    allNetwork[i] = (attackGraph, messageGraph, tradeGraph, allianceGraph, totalGraph) 

Day:  1
Day:  2
Day:  3
Day:  4
Day:  5
Day:  6
Day:  7
Day:  8
Day:  9
Day:  10
Day:  11
Day:  12
Day:  13
Day:  14
Day:  15
Day:  16
Day:  17
Day:  18
Day:  19
Day:  20
Day:  21
Day:  22
Day:  23
Day:  24
Day:  25
Day:  26
Day:  27
Day:  28
Day:  29
Day:  30


### This below cell will be use for any kind of check about nets, variables, indexes; for this reason no one comments are required

In [147]:
for edge in allNetwork[0][4].es:
    print(edge['Label'])


Trade
Trade
Trade
Trade
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Message
Message
Trade
Trade
Trade
Message
Message
Message
Message
Message
Message
Trade
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Atta

Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack

Message
Message
Message
Message
Message
Message
Message
Message
Message
Trade
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attac

Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack

Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack

...

Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Trade
Message
Message
Message
Message
Message
Message
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Attack
Attack
Attack
Attack
Attack
Message
Message
Message
Message
Message
Message
Message
Message
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Trade
Trade
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Attack
Atta

Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Attack
Attack
Attack
Attack
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Trade
Trade
Trade
Message
Message
Message
Message
Message
Message
Message
Trade
Trade
Trade
Trade
Attack
Attack
Attack
Attack
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Attack
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message

Message
Message
Message
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Message
Message
Message
Message
Message
Message
Message
Message
Message
Trade
Trade
Trade
Message
Message
Message
Message
Message
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Message
Message
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Message
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade


Trade
Trade
Trade
Trade
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Trade
Message
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Message
Message
Message
Message
Message
Message
Message
Message
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Message
Message
Message
Message
Message
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Message
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Message
Message
Message
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Message
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Trade
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Trade
Message
Trade
Trade
Trade
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Messag

Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message
Message


# START ANALYSIS !!!!

### Users number trend in the 30 days

In [146]:
nUsers = [0]*nGraph

for i in range(0, nGraph):
    nUsers[i] = allNetwork[i][4].vcount()

print("Users in the 30 days: ", nUsers)
print("")
print("{:>53}{:>7}.".format("Average users number in the 30 days is: ", sum(nUsers)/len(nUsers)))
print("{:>53}{:>7}.".format("The maximum number of users was recorded on the day: ", nUsers.index(max(nUsers))+1))

Users in the 30 days:  [3757, 3758, 3690, 3640, 3091, 3141, 3147, 3073, 3046, 2987, 3013, 2967, 2938, 2891, 2859, 2859, 2771, 2789, 2719, 2634, 2658, 2622, 2610, 2568, 2525, 2433, 2483, 2493, 2444, 2466]

             Average users number in the 30 days is:  2902.4.
The maximum number of users was recorded on the day:       2.


### Edges number trend in the 30 days

In [174]:
nAttack = [0]*nGraph
nTrade = [0]*nGraph
nMessage = [0]*nGraph

for i in range(0, nGraph):
    for edge in allNetwork[i][4].es:
        if edge['Label'] == "Attack":
            nAttack[i] += 1
        elif edge['Label'] == "Trade":
            nTrade[i] += 1
        elif edge['Label'] == "Message":
            nMessage[i] += 1
        else:
            print("This is a problem...")

print("{:>33}".format("Number of attacks in the 30 days:"), end = " ")
print(nAttack)
print("")
print("{:>32}".format("Number of trades in the 30 days:"), end = " ")
print(nTrade)
print("")
print("{:>34}".format("Number of messages in the 30 days:"), end = " ")
print(nMessage)
print("")
print("{:>40}{:>9}".format("Average number of attacks in 30 days:", sum(nAttack)/len(nAttack)))
print("{:>40}{:>9}".format("Average number of trades a in 30 days:", round(sum(nTrade)/len(nTrade), 1)))
print("{:>40}{:>9}".format("Average number of messages a in 30 days:", round(sum(nMessage)/len(nMessage), 1)))  
print("")
print("{:>40}{:>9}".format("Day with the highest number of attacks:", nAttack.index(max(nAttack))+1))
print("{:>40}{:>9}".format("Day with the highest number of trades:", nTrade.index(max(nTrade))+1))
print("{:>40}{:>9}".format("Average number of messages a in 30 days:", round(sum(nMessage)/len(nMessage), 1)))  

Number of attacks in the 30 days: [32997, 32782, 30264, 23883, 18748, 22424, 23448, 22550, 22401, 22778, 22162, 21628, 20675, 22086, 19969, 22134, 22367, 21772, 18820, 18390, 18771, 16970, 16477, 16467, 15715, 18164, 17230, 17057, 16513, 16749]

Number of trades in the 30 days: [9390, 8710, 8667, 8545, 9499, 9799, 9638, 8806, 8698, 8921, 9008, 9310, 9298, 9834, 10175, 9866, 9884, 9008, 8624, 8314, 8443, 8981, 8062, 8659, 8054, 8436, 9062, 8665, 8606, 8521]

Number of messages in the 30 days: [16346, 15019, 15316, 13274, 15337, 14612, 14454, 14619, 16082, 14643, 12486, 14447, 14597, 13144, 11805, 13474, 12239, 12888, 14883, 10589, 11405, 13685, 13178, 11773, 8277, 7957, 9341, 8869, 7587, 8468]

   Average number of attacks in 30 days:  21079.7
  Average number of trades a in 30 days:   8982.8
Average number of messages a in 30 days:  12693.1

  Day with the highest number of attacks        1
  Average number of trades a in 30 days:   8982.8
Average number of messages a in 30 days:  1269

# I prefer keep a bin cell rather than cry because i need already written code but deleted....
Of course in the final release this will be delete

In [ ]:
# for libraries install
!sudo apt install build-essential python-dev libxml2 libxml2-dev zlib1g-dev
!sudo pip install cairocffi
!pip install python-igraph
!pip install louvain

attackGraph = ig.Graph()
messageGraph = ig.Graph()
tradeGraph = ig.Graph()

totalPresent = False
alliancePresent = False

situa = 0

graphs = dict()
dayGraphs = list()

nGraph = 30

for i in range(0, 1):
    totalGraph = ig.Graph()
    
    messageGraph = messageGraph.Read_GraphML("../Dataset/Graphml-File/Messages-Network-Graphml/messages-timestamped-2009-12-"+str(i+1)+".graphml")
    for messageVertex in messageGraph.vs:
        if messageVertex.degree() != 0:
            allianceGraph.add_vertices(messageVertex['id'])
        totalPresent = False
        for totalVertex in totalGraph.vs:
            if messageVertex['id'] == totalVertex['name']:
                totalPresent = True
                break
        if not totalPresent:
            if messageVertex.degree() != 0:
                totalGraph.add_vertices(messageVertex['id'])
        totalPresent = False
    
    print("MESSAGE EDGE")        
    for edge in messageGraph.es:
        # edge['edgelabel'] = "Message"
        # print(situa, end = "")
        # print(", ", end = "")
        # situa += 1
        totalGraph.add_edge(messageGraph.vs[edge.tuple[0]], messageGraph.vs[edge.tuple[1]], edgelabel='Message')
        # allianceGraph.add_edge(edge.tuple[0], edge.tuple[1], edgelabel='Message')
    # situa = 0
    # print("")
        

    

    print("MESSAGE GRAPH:", messageGraph.summary())
    print("TOTAL GRAPH:", totalGraph.summary())
    
    print("")
    
    for vertex in totalGraph.vs:
    if vertex['name'] == '9152':
        print(vertex.outdegree())
attackGraph = ig.Graph()
messageGraph = ig.Graph()
tradeGraph = ig.Graph()

totalPresent = False
alliancePresent = False

situa = 0

graphs = dict()
dayGraphs = list()

nGraph = 30

for i in range(0, nGraph):
    totalGraph = ig.Graph()
    allianceGraph = ig.Graph()
    print("Graph number: ", (i + 1))
    
    attackGraph = attackGraph.Read_GraphML("../Dataset/Graphml-File/Attacks-Network-Graphml/attacks-timestamped-2009-12-"+str(i+1)+".graphml")
    for vertex in attackGraph.vs:
        totalGraph.add_vertices(vertex['id'])
        # print(vertex['id'] +", ", end="")
    # print("")
    print("ATTACK EDGE")
    for edge in attackGraph.es:
        # edge['edgelabel'] = "Attack"
        # print(situa, end = "")
        # print(", ", end = "")
        # situa += 1
        totalGraph.add_edge(attackGraph.vs[edge.tuple[0]], attackGraph.vs[edge.tuple[1]], edgelabel='Attack') 
    situa = 0
    # print("")
    
    messageGraph = messageGraph.Read_GraphML("../Dataset/Graphml-File/Messages-Network-Graphml/messages-timestamped-2009-12-"+str(i+1)+".graphml")
    for messageVertex in messageGraph.vs:
        allianceGraph.add_vertices(messageVertex['id'])
        totalPresent = False
        for totalVertex in totalGraph.vs:
            if messageVertex['id'] == totalVertex['name']:
                totalPresent = True
                break
        if not totalPresent:
            totalGraph.add_vertices(messageVertex['id'])
        totalPresent = False
    
    print("MESSAGE EDGE")        
    for edge in messageGraph.es:
        # edge['edgelabel'] = "Message"
        # print(situa, end = "")
        # print(", ", end = "")
        # situa += 1
        totalGraph.add_edge(messageGraph.vs[edge.tuple[0]], messageGraph.vs[edge.tuple[1]], edgelabel='Message')
        allianceGraph.add_edge(messageGraph.vs[edge.tuple[0]], messageGraph.vs[edge.tuple[1]], edgelabel='Message')
    # situa = 0
    # print("")
        
    tradeGraph = tradeGraph.Read_GraphML("../Dataset/Graphml-File/Trades-Network-Graphml/trades-timestamped-2009-12-"+str(i+1)+".graphml")
    for tradeVertex in tradeGraph.vs:
        totalPresent = False
        for totalVertex in totalGraph.vs:
            if tradeVertex['id'] == totalVertex['name']:
                totalPresent = True
                break
        if not totalPresent:
            totalGraph.add_vertices(tradeVertex['id'])
        
    for tradeVertex in tradeGraph.vs:
        alliancePresent = False
        for allianceVertex in allianceGraph.vs:
            if tradeVertex['id'] == allianceVertex['name']:
                alliancePresent = True
                break
        if not alliancePresent:
            allianceGraph.add_vertices(tradeVertex['id'])
        
    
    print("TRADE EDGE")
    for edge in tradeGraph.es:
        # edge['edgelabel'] = "Trade"
        # print(situa, end = "")
        # print(", ", end = "")
        # situa += 1
        totalGraph.add_edge(tradeGraph.vs[edge.tuple[0]], tradeGraph.vs[edge.tuple[1]], edgelabel='Trade')
        allianceGraph.add_edge(tradeGraph.vs[edge.tuple[0]], tradeGraph.vs[edge.tuple[1]], edgelabel='Trade')
    situa = 0
    # allianceGraph = messageGraph.union(tradeGraph)
    # totalGraph = allianceGraph.union(attackGraph)
    # print("")
    
    print("ATTACK GRAPH:", attackGraph.summary())
    print("MESSAGE GRAPH:", messageGraph.summary())
    print("TRADE GRAPH:", tradeGraph.summary())
    print("ALLIANCE GRAPH:", allianceGraph.summary())
    print("TOTAL GRAPH:", totalGraph.summary())
    totalGraph.save("../Dataset/Graphml-File/Total-Network-Graphml/total-2009-12-"+str(i+1)+".graphml")
    allianceGraph.save("../Dataset/Graphml-File/Alliance-Network-Graphml/alliance-2009-12-"+str(i+1)+".graphml")
    print("")
    






'''G = ig.Graph()
G = G.Read_GraphML("../Dataset/Graphml-File/Trades-Network-Graphml/trades-timestamped-2009-12-1.graphml") 
G += G.Read_GraphML("../Dataset/Graphml-File/Messages-Network-Graphml/messages-timestamped-2009-12-1.graphml") 
G += G.Read_GraphML("../Dataset/Graphml-File/Attacks-Network-Graphml/attacks-timestamped-2009-12-1.graphml") 
print(G)'''


# print(G.vs.attribute_names())
# print(G.es.attribute_names())
# print(G.degree())
# print(G.indegree())
# print(G.outdegree())

# totalGraph = totalGraph.Read_GraphML("../Dataset/Graphml-File/Total-Network-Graphml/total-2009-12-1.graphml")
allianceGraph = ig.Graph()
allianceGraph = allianceGraph.Read_GraphML("../Dataset/Graphml-File/Alliance-Network-Graphml/alliance-2009-12-1.graphml")

totalGraph = ig.Graph()
totalGraph = totalGraph.Read_GraphML("../Dataset/Graphml-File/Total-Network-Graphml/total-2009-12-1.graphml")

attackGraph = ig.Graph()
attackGraph = attackGraph.Read_GraphML("../Dataset/Graphml-File/Attacks-Network-Graphml/attacks-timestamped-2009-12-1.graphml")

messageGraph = ig.Graph()
messageGraph = messageGraph.Read_GraphML("../Dataset/Graphml-File/Messages-Network-Graphml/messages-timestamped-2009-12-1.graphml") 

tradeGraph = ig.Graph()
tradeGraph = tradeGraph.Read_GraphML("../Dataset/Graphml-File/Trades-Network-Graphml/trades-timestamped-2009-12-1.graphml")

# print(totalGraph.degree())
# print(messageGraph.degree())
# print(attackGraph.degree())
# print(tradeGraph.degree())
print("AllianceGraph vertex with 0 degree")
for vertex in allianceGraph.vs:
    if vertex.degree() == 0:
        print(vertex['id'] +", ", end = "")
print("")
print("")
print("TotalGraph vertex with 0 degree")
for vertex in totalGraph.vs:
    if vertex.degree() == 0:
        print(vertex['id']+", ", end="")
print("")
print("")

for vertex in attackGraph.vs:
    if vertex.degree() == 0:
        print(vertex['id']+", ", end="")

print("")
print("")
ok = False
print("Attack Graph Check in Total Graph")
for vertex1 in attackGraph.vs:
    ok = False
    if vertex1['id'] == 'n2807':
        print("i am here 1")
    for vertex2 in totalGraph.vs:
        if vertex1['id'] == vertex2['name']:
            ok = True
            break
    if not ok:
        print("error")

print("")
print("")
ok = False
print("Message Graph Check in Total Graph")
for vertex1 in messageGraph.vs:
    ok = False
    if vertex1['id'] == 'n2807':
        print("i am here 2")
    for vertex2 in totalGraph.vs:
        if vertex1['id'] == vertex2['name']:
            ok = True
            break
    if not ok:
        print("error")
        
print("")
print("")
ok = False
print("Trade Graph Check in Total Graph")
for vertex1 in tradeGraph.vs:
    if vertex1['id'] == 'n2807':
        print("i am here 3")
    ok = False
    for vertex2 in totalGraph.vs:
        if vertex1['id'] == vertex2['name']:
            ok = True
            break
    if not ok:
        print("error")

print("")
print("")
ok = False
print("Message Graph Check in Alliance Graph")
for vertex1 in messageGraph.vs:
    ok = False
    for vertex2 in allianceGraph.vs:
        if vertex1['id'] == vertex2['name']:
            ok = True
            break
    if not ok:
        print("error")
        
print("")
print("")
ok = False
print("Trade Graph Check in Alliance Graph")
for vertex1 in tradeGraph.vs:
    ok = False
    for vertex2 in allianceGraph.vs:
        if vertex1['id'] == vertex2['name']:
            ok = True
            break
    if not ok:
        print("error")
    
print(allianceGraph.summary())
print(totalGraph.summary())

for vertex in totalGraph.vs:
    if vertex['id'] == 'n2807':
        print(vertex)
print("")
print("show in the graph")
for vertex in totalGraph.vs:
    if vertex['name'] == '9152':
        print(vertex)
    
print("In Total Graph")
for edge in totalGraph.es:
    if edge.tuple[0] == '9152' or edge.tuple[1] == '9152':
        print(edge)

print("In Message Graph")
for edge in messageGraph.es:
    if edge.tuple[0] == '9152':
        print(edge)
    if edge.tuple[1] == '9152':
        print(edge)
print("")
for vertex in attackGraph.vs:
    if vertex['id'] == '9152':
        print("attack")
        print(vertex)

for vertex in tradeGraph.vs:
    if vertex['id'] == '9152':
        print("trade")
        print(vertex)


for vertex in messageGraph.vs:
    if vertex['id'] == '9152':
        print("message")
        print(vertex)

        
list(attackGraph.vs)

import networkx as nx

messageGraph = nx.MultiDiGraph()
tradeGraph = nx.MultiDiGraph()
attackGraph = nx.MultiDiGraph()
allianceGraph = nx.MultiDiGraph()
totalGraph = nx.MultiDiGraph()

realMessage = nx.MultiDiGraph()
realTrade = nx.MultiDiGraph()
realAttack = nx.MultiDiGraph()

messageGraph = nx.read_graphml("../Dataset/Graphml-File/Messages-Network-Graphml/messages-timestamped-2009-12-1.graphml")
tradeGraph = nx.read_graphml("../Dataset/Graphml-File/Trades-Network-Graphml/trades-timestamped-2009-12-1.graphml")
attackGraph = nx.read_graphml("../Dataset/Graphml-File/Attacks-Network-Graphml/attacks-timestamped-2009-12-1.graphml")
# nx.info(messageGraph)

# print(messageGraph.nodes()[0])

# for edge in messageGraph.edges():   
#    messageGraph[edge[0]][edge[1]].update(label= "Message")

# nx.set_edge_attributes(messageGraph, "Label", "Message")
# nx.set_edge_attributes(tradeGraph, "Label", "Trade")
# nx.set_edge_attributes(attackGraph, "Label", "Attack")

# allianceGraph = nx.disjoint_union(messageGraph, tradeGraph)

# nx.set_edge_attributes(G, bb, 'betweenness')

# for itr in allianceGraph.edges_iter(None, True, True):
#        print(itr)



messageGraph.edges(data=True,keys=True)

for itr in messageGraph.edges_iter(None, True, True):
    realMessage.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"m", Label= "Message")
    
for itr in tradeGraph.edges_iter(None, True, True):
    realTrade.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"t", Label= "Trade")

for itr in attackGraph.edges_iter(None, True, True):
    realAttack.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"a", Label= "Attack")
    

allianceGraph = nx.compose(realMessage, realTrade)
totalGraph = nx.compose(allianceGraph, realAttack)
print("MESSAGE GRAPH: ", nx.info(messageGraph))
print("")
print("TRADE GRAPH: ", nx.info(tradeGraph))
print("")
print("ATTACK GRAPH: ", nx.info(attackGraph))
print("")
print("ALLIANCE GRAPH: ", nx.info(allianceGraph))
print("")
print("TOTAL GRAPH: ", nx.info(totalGraph))

# print(allianceGraph.degree())
print("")
print("")
# print(totalGraph.degree())

nx.write_graphml(realMessage, "../Dataset/Graphml-File/Total-Network-Graphml/total-2009-12-30.graphml")

messageGraph = ig.Graph()
messageGraph = messageGraph.Read_GraphML("../Dataset/Graphml-File/Total-Network-Graphml/total-2009-12-30.graphml") 

print(messageGraph.summary())

# for edge in messageGraph.es:
#    print(edge)
    
#for vertex in messageGraph.vs:
#    print(vertex)


